In [9]:
from PIL import Image, ImageFilter
import numpy as np
import cv2
from matplotlib import pyplot as plt

In [10]:
def detectWhite( pix ):
    threshold = 130
    if pix[0] > threshold and pix[1] > threshold and pix[2] > threshold:
        return True
    return False

In [11]:
def WhitenImage( imagefilepath ):
    im = Image.open( imagefilepath )
    #im = im.filter(ImageFilter.BLUR)
    pix = im.load()
    width, height = im.size   # Get dimensions
    #print('Image Width: %d, Height: %d' %(width, height))

    # Paper region detection
    # Note: requires picture taken on blue background
    for row in range(height):
        for col in range( width):
            if detectWhite(pix[col,row]):
                pix[col,row] = (255,255,255)
    im.save( 'W_'+imagefilepath , "JPEG")
    return 'W_'+imagefilepath

In [12]:
def detectBlue( pix ):
    if pix[2] > pix[1]*1.2 and pix[2] > pix[0]*1.2:
        return True
    return False

In [13]:
def t10_transform(frame,imgfile_in,imgfile_out):
    #upperleft_x,upperleft_y,upperright_x,upperright_y,lowerleft_x,lowerleft_y,lowerright_x ,lowerright_y
    img = cv2.imread(imgfile_in)
    w = 100
    h = 1100
    new_img = np.float32([[0, 0], [w, 0], [0, h], [w, h]])
    old_img_cp = np.float32([frame[:2],frame[2:4],frame[4:6],frame[6:]])
    matrix = cv2.getPerspectiveTransform(old_img_cp, new_img)
    result = cv2.warpPerspective(img, matrix, (w, h))
    #cv2.imshow("Old_Image", img)
    #cv2.imshow("New_Image", result)
    cv2.imwrite(imgfile_out,result)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [16]:
##### Read image
def PreProcessShredImage( imagefilepath ):
    im = Image.open( imagefilepath )
    im = im.filter(ImageFilter.BLUR)
    pix = im.load()
    width, height = im.size   # Get dimensions
    print('Image Width: %d, Height: %d' %(width, height))

    # Paper region detection
    # Note: requires picture taken on blue background
    for row in range(height):
        for col in range( width):
            if detectBlue(pix[col,row]):
                pix[col,row] = (0,0,255)
            else:
                pix[col,row] = (255,255,255)

    # find side slopes
    x1 = []
    x2 = []
    heightA = height//5
    heightB = 4*height//5
    for col in range(width-1):
        if pix[col, heightA] != pix[col+1,heightA]:
            x1.append(col)
        if pix[col, heightB] != pix[col+1,heightB]:
            x2.append(col)

    print('top x crossings:',x1)
    print('bottom x crossings:', x2)
    sideslopes = []
    sidebees = []
    for i in range(len(x1)):
        try:
            slope = (3*height//5)/(x2[i]-x1[i])
        except:
            slope = (3*height//5)
        bee = (height//5)- slope*x1[i]
        sideslopes.append(slope)
        sidebees.append(bee)

    #print(slopes)
    #print(bees)

    topslopes = []
    topBees = []
    # find a top/bottom slope
    for i in range(0,16,2):
        xA = x1[i]+30
        xB = x1[i+1]-45
        yA = []
        yB = []
        for row in range(height-1):
            if pix[xA,row] != pix[xA,row+1]:
                yA.append(row)
            if pix[xB,row] != pix[xB,row+1]:
                yB.append(row)
        slope = (yB[0]-yA[0])/(xB-xA)
        b = yA[0]-slope*xA
        topslopes.append(slope)
        topBees.append(b)
        #print("ya/yb",yA[0],yB[0])    

    botslopes = []
    botBees = []
    # find a top/bottom slope
    for i in range(0,16,2):
        xA = x2[i]+45
        xB = x2[i+1]-35
        yA = []
        yB = []
        for row in range(4*height//5,height-1):
            if pix[xA,row] != pix[xA,row+1]:
                yA.append(row)
            if pix[xB,row] != pix[xB,row+1]:
                yB.append(row)
        slope = (yB[0]-yA[0])/(xB-xA)
        b = yA[0]-slope*xA
        botslopes.append(slope)
        botBees.append(b)
        #print("ya/yb",yA[0],yB[0])    

    # painting red edges (sides)
    for y in range(height):
        for i in range(len(sideslopes)):
            x = int((y - sidebees[i])/sideslopes[i])
            #print('*',x,y)
            pix[x,y] = (255,0,0)        


    # painting black edge (top)
    topcorners = []
    botcorners = []
    toggle = True
    for i in range(0,16,2):
        for x in range(x1[i]-40,x1[i+1]+40):
            y = int(topslopes[i//2]*x + topBees[i//2])
            if pix[x,y] == (255,0,0):
                if toggle:
                    #print(">TOP LEFT %d CORNER FOUND: (%d/%d)" %(i//2,x,y))
                    toggle = False
                else:
                    #print(">TOP RIGHT %d CORNER FOUND: (%d/%d)" %(i//2,x,y))
                    toggle = True
                topcorners.append((x,y))
                pix[x,y] = (0,255,255)
            else:
                pix[x,y] = (0,0,0)

    # painting black edge (bottom)
    for i in range(0,16,2):
        for x in range(x2[i]-40,x2[i+1]+60):
            y = int(botslopes[i//2]*x + botBees[i//2])
            if pix[x,y] == (255,0,0):
                if toggle:
                    #print(">BOT LEFT %d CORNER FOUND: (%d/%d)" %(i//2,x,y))
                    toggle = False
                else:
                    #print(">BOT RIGHT %d CORNER FOUND: (%d/%d)" %(i//2,x,y))
                    toggle = True
                botcorners.append((x,y))
                pix[x,y] = (0,255,255)
            else:
                pix[x,y] = (0,0,0)

    imageFrames = []
    # need a list of lists in the form of [topLx,topLy,topRx,topRy,lowLx,lowLy,LowRx,LowRy]
    for i in range(0,len(topcorners),2):
        frame = [topcorners[i][0],topcorners[i][1],topcorners[i+1][0],topcorners[i+1][1],  
                 botcorners[i][0],botcorners[i][1],botcorners[i+1][0],botcorners[i+1][1]]
        imageFrames.append(frame) 

    #im.show()
    #for frame in imageFrames:
    #    print(frame)
    return imageFrames

In [30]:
def Main( imagename ):
    iFrames = PreProcessShredImage(imagename)
    #iFrames
    count = 1
    image = WhitenImage(imagename)
    subImages = []
    
    print('Neuralnetting...')
    
    for frame in iFrames:
        newFile = image.split('.')[0]+('_%d.jpg' %(count))
        t10_transform(frame, image, newFile )
        subImages.append(newFile)
        count += 1
    
    print('Reassembling...')
    
    im1 = Image.open( subImages[1] )
    widthX, heightY = im1.size
    im2 = Image.open( subImages[3] )
    im3 = Image.open( subImages[0] )
    im4 = Image.open( subImages[7] )
    im5 = Image.open( subImages[6] )
    im6 = Image.open( subImages[4] )
    im7 = Image.open( subImages[5] )
    im8 = Image.open( subImages[2] )
    dst = Image.new('RGB', (widthX*8, heightY))
    dst.paste(im1, (0, 0))
    dst.paste(im2, (im1.width, 0))
    dst.paste(im3, (2*im1.width, 0))
    dst.paste(im4, (3*im1.width, 0))
    dst.paste(im5, (4*im1.width, 0))
    dst.paste(im6, (5*im1.width, 0))
    dst.paste(im7, (6*im1.width, 0))
    dst.paste(im8, (7*im1.width, 0))
    dst.save( 'FixedSolution_%s' %(imagename) , "JPEG")
    
    print('Done.')
    
    #dst.show()
    #return dst
    return 'FixedSolution_%s' %(imagename)
    
    # TODO : Format the image strips (currently 8 files) into a format that matches 
    # neural net thingy
    # return some numpy array bundle
    # OR save the numpy array package and return package name

In [31]:
Main('Shredded04.jpg')

Image Width: 5312, Height: 2988
top x crossings: [1330, 1590, 1692, 1942, 2082, 2330, 2431, 2672, 2767, 3012, 3105, 3352, 3486, 3736, 3875, 4119]
bottom x crossings: [1258, 1522, 1623, 1880, 2058, 2317, 2401, 2651, 2767, 3024, 3151, 3409, 3504, 3764, 3915, 4172]
Neuralnetting...
Reassembling...
Done.
